# Model Fitting

We want to find the causal effect of studying on grades, so we will be using some econometric techniques, focusing on causal inference. We will first run a naive OLS fit, and then demonstrate why it is inappropriate in this context.

### Naive OLS Fit

The naive approach would be to use these data to fit a "kitchen sink" OLS regression to the data. So lets see what this regression would yield, and then address the plausibility of these results.

*Note: We are using MacKinnon and White's (1985) HC3 heteroskedasticity robust covariance estimator*

In [10]:
# Loading the libraries we will use and setting global options

# Data manipulation and math/stats functions
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import statsmodels.api as sm
#!pip install linearmodels
from linearmodels.iv import IV2SLS 

# Plotting preferences
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

# Suppressing warnings
import warnings
warnings.simplefilter(action = "ignore")

In [11]:
# Loading the data
student_perf = pd.read_pickle('data/student_por_v2.pkl')

# Data formatting - converting strings to indicators
student_perf['school_GP']  = 1*(student_perf.school == 'GP')
student_perf['male']       = 1*(student_perf.sex == 'M')
student_perf['urban']      = 1*(student_perf.address == 'U')
student_perf['fam_small']  = 1*(student_perf.famsize == 'LE3')
student_perf['fam_split']  = 1*(student_perf.Pstatus == 'A')
student_perf['no_parent']  = 1*(student_perf.guardian == 'other')
student_perf['father']     = 1*(student_perf.guardian == 'father')
student_perf['mother']     = 1*(student_perf.guardian == 'mother')
student_perf['school_sup'] = 1*(student_perf.schoolsup == 'yes')
student_perf['famsup']     = 1*(student_perf.famsup == 'yes')
student_perf['paid']       = 1*(student_perf.paid == 'yes')
student_perf['activities'] = 1*(student_perf.activities == 'yes')
student_perf['nursery']    = 1*(student_perf.nursery == 'yes')
student_perf['higher']     = 1*(student_perf.higher == 'yes')
student_perf['internet']   = 1*(student_perf.internet == 'yes')
student_perf['romantic']   = 1*(student_perf.romantic == 'yes')

# Converting G3 to percent
student_perf['G3_perc']    = student_perf.G3 / 12

In [12]:
# Running the OLS model
Y = student_perf.G3_perc
X = student_perf[['studytime', 'school_GP', 'male', 'age', 'urban', 'fam_small', 'fam_split', 'Medu', 'Fedu', 'no_parent','mother', 'father', 'traveltime', 'freetime', 'failures', 'school_sup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'goout', 'Dalc', 'Walc', 'health', 'absences']]
X = sm.add_constant(X)
model = sm.OLS(Y, X)
results = model.fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                G3_perc   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     648.2
Date:                Thu, 30 Nov 2017   Prob (F-statistic):               0.00
Time:                        11:48:58   Log-Likelihood:                 69.417
No. Observations:                 649   AIC:                            -80.83
Df Residuals:                     620   BIC:                             48.95
Df Model:                          28                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4336      0.121      3.572      0.0

These results suggest that within this population, an additional hour of studying per week would "result" in an increase of 3 percentage points to the final grade (with high statistical significance). But this conclusion is not necessarily the correct one.

Many of the coefficients are not statistically significant, and our X matrix is close to singular (suggesting a problem with multicollinearity as `statsmodels.api` explains). While this is not necessarily a cause for concern, it might cast some suspicion on the results.

This result is unsurprising because in this regression we include proxies to represent unobserved ability, and include our variable of interest, hours of studying. This set up makes the implicit assumption that hours of studying and ability are independent. Unforunately, the validity of this assumption is questionable. It is certainly plausible that students with higher abilities enjoy studying more (perahps because they find it less difficult), suggesting that they will have a less negative coefficient on studying in their utility maximizations, leading them to study for longer. So the assumption that hours of studying and ability are independent is quite strong, and not necessarily valid. Violation of this assumption would suggest that the dependence between ability and studying is a source of serious multicollinearity. On the other hand, our low $\mathrm{R}^2$ suggests that multicollinearity might not be an issue here; and that we probably just need more data. We could try to address this potential multicollinearity by dropping some variables, but generally it is better to report the full model and point out the potential issue. We don't usually drop variables because that could cause a more agregious problem through confounding effects. The ideal solution here would be to collect more data, but that is not a possibility. While multicollinearity might be present, we can't do much about it.

There is another, more serious, issue with this naive approach of running an OLS fit on the data - endogeneity. In our model we have attempted to use proxies to identify ability, but what if our proxies are not fully identifying ability?  Then clearly the covariance between our $x_k$'s and the error terms is non-zero: we have endogenous variables. This endogeneity means that OLS is not a consistent estimator for our vector of $\beta$'s! We have the following:
$$
\mathrm{b}_k \stackrel{p}{\longrightarrow} \beta_k + \gamma \frac{\mathrm{Cov}[q, x_k]}{\mathrm{Var}[x_k]}
$$
Where $q$ is the unidentified portion of ability, $\gamma$ is the true coefficient on $q$ (if we could identify it), $\mathrm{b}_k$ is the OLS estimated coefficient on $x_k$, and $\beta_k$ is the true coefficient on $x_k$. Notice that $q$ is determined by how well our proxies identify ability. The better our proxies identify ability, the smaller the bias term $\gamma \frac{\mathrm{Cov}[q, x_k]}{\mathrm{Var}[x_k]}$ will be (with bias arising from our endogeneity problem).

In many settings, researchers assume that $\mathrm{Cov}[q, x_k] = 0$ except for the variable of interest (in our case, hours of studying). This is also a plausible assumption in our setting; even more so when we consider that we already determined that study time and ability might be dependent (and that part of ability is in the error term). So we have simplified our problem to identification of the true coefficient on the endogenous variable hours of studying. We should note that since we do not know what part of "ability" is being omitted, we cannot make any guesses about which way our OLS estimated coefficient is biased (because we can't determine the sign of $\mathrm{Cov}[q, studying]$ ).

### Addressing Endogeneity

Now that we have identified our problem, we need to find a way to account for it. Specifically, we need to find one or more instruments for hours of studying. A good instrument, $z$, would have the following characteristics:
* $\mathrm{Cov}[z, studying] \ne 0 \ \quad$ (hours of studying covaries with $z$)
* $\mathrm{Cov}[z, \varepsilon] = 0 \ \ \ \quad\quad\quad$ ($z$ is exogenous)

Finding valid instruments proves to be fairly challenging, because we can often imagine situations where a potential instrument is not valid. For example, we might be tempted to include `school_sup` (extra educational support) as an instrument because if someone is enrolled in extra support they will study more. But upon further thought, it seems likely that enrollment in tutoring is not independent of ability (perhaps high or low ability students are more likely to get tutored), meaning that perhaps `school_sup` covaries with the unidentified ability$^1$.

After some thought I concluded that `school_GP` (which school the student is enrolled in), `goout` (how often the student goes out with friends), and `male` (indicator for male), are valid instruments for hours of studying. School enrollment can be a valid instrument because studying habits are often modelled in networks$^2$; when one student with high centrality increases their studying, connected students (friends or classmates) also increase their studying. It is easy to imagine that the two schools have friendship networks that have fewer heterophilic links than homophilic links, which implies that hours of studying covaries with enrollment through random perturbations in central students' studying habits. Frequency of going out is a valid instrument because it seems likely that most students go out with friends, and that students who go out more have less time to study. We can assume that going out and ability are independent because there is no reason to think that going out would covary with ability. Lastly, gender is a valid instrument for a similar reason to that of enrollment; male to male links (friendships) will be more common than male to female links, so if central males study less than central females, then we would expect males to study less than females on average (and vice versa).

Now that we have some plausible instruments for our endogenous variable, we can run 2SLS to estimate the model. Note that we exclude `no_parent` (trying to address some multicollinearity from splitting categorical variables into multiple indicators) from the exogenous variables. We are again using a heteroskedasticity robust variance-covariance matrix estimator - because robustness can never hurt.

-----
1. Earlier we had assumed that all variables other than study time were exogenous, meaning that any of those variables that was correlated with study time would be a valid instrument. But the assumption we made was very weak, in reality we don't really care if variables other than study time are endogenous because we are not interested in the coefficient on these variables. So when we are thinking about constructing instruments for schooling we still need to critically consider the exogeneity of the instrument.
2. This specific example is from Bryan Graham's course Econ 142 during a discussion of networks and centrality. 

In [13]:
# Running the 2SLS model

# Dependent variable
Y = student_perf.G3_perc

# Exogenous variables
X_exog = student_perf[['age', 'urban', 'fam_small', 'fam_split', 'Medu', 'Fedu', 'mother', 'father', 'traveltime', 'freetime', 'failures', 'school_sup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'Dalc', 'Walc', 'health', 'absences']]
X_exog = sm.add_constant(X_exog)

# Endogenous variable
X_endog = student_perf.studytime

# Instruments
Z = student_perf[['goout', 'male']]

model = IV2SLS(Y, X_exog, X_endog, Z)
results = model.fit()
print(results.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                G3_perc   R-squared:                     -0.5445
Estimator:                    IV-2SLS   Adj. R-squared:                -0.6065
No. Observations:                 649   F-statistic:                    154.12
Date:                Thu, Nov 30 2017   P-value (F-stat)                0.0000
Time:                        11:51:35   Distribution:                 chi2(25)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.3895     0.2798     1.3921     0.1639     -0.1589      0.9379
age           -0.0108     0.0143    -0.7571     0.44

These results are very encouraging. Usually we expect the significance of the coefficients to decrease for almost all the variables in a 2SLS model compared to the analog OLS model. In our 2SLS fit we find that most of the coefficients are indeed less significant, but that the coefficient on study time is actually more significant by an order of magnitude (0.0003 vs 0.002)! Not only that, but the coefficient itself is an order of magnitude larger (0.36 vs 0.03)! The increase in significance of the coefficient on our endogenous variable is highly suggestive that we have good instruments, and better identification of the marginal effect of studying on grades. 

One item to note is that in the 2SLS fit, our value for $R^2$ is negative. It turns out that in 2SLS (and 3SLS), a negative $R^2$ is not usually a cause for concern. In instrumental variable models we are more interested in the standard error and significance level on the coefficient of our endogenous variable. And in our fit, the coefficient is highly significant. For a more in depth analysis of the negative $R^2$ phenomenon, Stata documention provides a good explanation [here](http://www.stata.com/support/faqs/statistics/two-stage-least-squares/ "2SLS R^2").

### Conclusions

After addressing our issues of endogeneity, we end up with a model that is plausibly identifying the marginal effect of studying on grades. Our results estimate that a marginal increase in hours of studying (per week) results in a 36 point increase in final grade (as a percent). We must keep in mind the limitations of this model; the limited sample greatly reduce our ability to make general statements about this finding. However, our results are still indicative that in secondary schools, additional studying causes significant increases to final grades. 

A direction for future study would include an increase to the complexity of the model. It seems unlikely that three hours of studying per week is be enough to ensure almost anyone gets a perfect grade - so perhaps we need to allow for a quadratic (or higher degree) relationship between studying and grades.

### References

Card, D., & Krueger, A. (1992). Does School Quality Matter? Returns to Education and the Characteristics of Public Schools in the United States. *Journal of Political Economy*, 100(1), 1-40.

Cortez, P. and Silva, A. Using Data Mining to Predict Secondary School Student Performance. In A. Brito and J. Teixeira Eds., Proceedings of 5th FUture BUsiness TEChnology Conference (FUBUTEC 2008) pp. 5-12, Porto, Portugal, April, 2008, EUROSIS, ISBN 978-9077381-39-7. 

Greene, W. H. (2000). Econometric analysis. Upper Saddle River, N.J: Prentice Hall.

MacKinnon, J.G. and H. White. (1985), Some heteroskedasticity consistent covariance matrix estimators with improved finite sample properties. *Journal of Econometrics*, 29, 53-57.